In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [ ]:
%%writefile Multi.cu
#include <cuda_runtime.h>
#include <iostream>
__global__ void matmul(int* A, int* B, int* C, int N) {
int Row = blockIdx.y*blockDim.y+threadIdx.y;
int Col = blockIdx.x*blockDim.x+threadIdx.x;
if (Row < N && Col < N) {
int Pvalue = 0;
for (int k = 0; k < N; k++) {
Pvalue += A[Row*N+k] * B[k*N+Col];
}
C[Row*N+Col] = Pvalue;
}
}
int main() {
int N = 100;
int size = N * N * sizeof(int);
int* A, * B, * C;
int* dev_A, * dev_B, * dev_C;
cudaMallocHost(&A, size);
cudaMallocHost(&B, size);
cudaMallocHost(&C, size);
cudaMalloc(&dev_A, size);
cudaMalloc(&dev_B, size);
cudaMalloc(&dev_C, size);
// Initialize matrices A and B
for (int i = 0; i < N; i++) {
for (int j = 0; j < N; j++) {
A[i*N+j] = i*N+j;
B[i*N+j] = j*N+i;
}
}
cudaMemcpy(dev_A, A, size,
cudaMemcpyHostToDevice);
cudaMemcpy(dev_B, B, size,
cudaMemcpyHostToDevice);
dim3 dimBlock(16, 16);
dim3 dimGrid(N/dimBlock.x, N/dimBlock.y);
matmul<<<dimGrid, dimBlock>>>(dev_A, dev_B,dev_C, N);
cudaMemcpy(C, dev_C, size, cudaMemcpyDeviceToHost);
// Print the result
for (int i = 0; i < 10; i++) {
for (int j = 0; j < 10; j++) {
std::cout << C[i*N+j] << " ";
}
std::cout << std::endl;
}
// Free memory
cudaFree(dev_A);
cudaFree(dev_B);
cudaFree(dev_C);
cudaFreeHost(A);
cudaFreeHost(B);
cudaFreeHost(C);
return 0;
}

Overwriting Multi.cu


In [ ]:
!nvcc Multi.cu -o Multi

In [ ]:
!./Multi

328350 823350 1318350 1813350 2308350 2803350 3298350 3793350 4288350 4783350 
823350 2318350 3813350 5308350 6803350 8298350 9793350 11288350 12783350 14278350 
1318350 3813350 6308350 8803350 11298350 13793350 16288350 18783350 21278350 23773350 
1813350 5308350 8803350 12298350 15793350 19288350 22783350 26278350 29773350 33268350 
2308350 6803350 11298350 15793350 20288350 24783350 29278350 33773350 38268350 42763350 
2803350 8298350 13793350 19288350 24783350 30278350 35773350 41268350 46763350 52258350 
3298350 9793350 16288350 22783350 29278350 35773350 42268350 48763350 55258350 61753350 
3793350 11288350 18783350 26278350 33773350 41268350 48763350 56258350 63753350 71248350 
4288350 12783350 21278350 29773350 38268350 46763350 55258350 63753350 72248350 80743350 
4783350 14278350 23773350 33268350 42763350 52258350 61753350 71248350 80743350 90238350 
